This example requires you to get an OPENAI_API_KEY. 


In [1]:
from zmongo_retriever.zmongo_toolbag.zmongo.BAK import ZMongoRetriever
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.llamacpp import LlamaCpp
from langchain.chains import load_summarize_chain
from zmongo_retriever.zmongo_toolbag.zmongo import zconstants

Set your variables

In [2]:
model_path = zconstants.MODEL_PATH
print(model_path)
 # Your .gguf file path
mongo_db_name = zconstants.MONGO_DATABASE_NAME # Your MongoDB database name
mongo_uri = zconstants.MONGO_URI # Your mongo_uri
this_collection_name = zconstants.DEFAULT_COLLECTION_NAME  # Your MongoDB collection
this_page_content_key = 'casebody.data.opinions.0.text'  # Specify the field to use as page_content
this_document_id = '65f1b6beae7cd4d4d1d3ae8d'  # Example ObjectId('_id') value

/media/overlordx/DATA/_models/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_0.gguf


Use ZMongoRetriever to split the text from the page_content_field into LangChain Documents

In [3]:
# larger values for chunk_size may solve problems with exceeding your token limit
retriever = ZMongoRetriever(mongo_uri=mongo_uri, chunk_size=1000, collection_name=this_collection_name, page_content_field=this_page_content_field)
documents_by_id = retriever.invoke(ojbect_ids=this_document_id, page_content_key=this_page_content_key)
print(documents_by_id)

[[<zmongo_retriever.Document object at 0x72e5d9bfc070>, <zmongo_retriever.Document object at 0x72e5d9bfc0d0>, <zmongo_retriever.Document object at 0x72e5d9bfc130>, <zmongo_retriever.Document object at 0x72e5d9bfc190>, <zmongo_retriever.Document object at 0x72e5d9bfc1f0>, <zmongo_retriever.Document object at 0x72e5d9bfc250>, <zmongo_retriever.Document object at 0x72e5d9bfc2b0>, <zmongo_retriever.Document object at 0x72e5d9bfc310>]]


If the output from above == [] then you have a problem with your variables.

Use LlamaCPP to summarize the chunked output.

In [4]:
prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=0,
    n_gpu_layers=-1,
    n_ctx=4096,
    n_batch=4096,
    verbose=True,
    f16_kv=True
)
stuff_chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
summary_chain_result = stuff_chain.invoke({'input_documents': documents_by_id[0]})
print(summary_chain_result)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /media/overlordx/DATA/_models/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 12

{'input_documents': [<zmongo_retriever.Document object at 0x72e5d9bfc070>, <zmongo_retriever.Document object at 0x72e5d9bfc0d0>, <zmongo_retriever.Document object at 0x72e5d9bfc130>, <zmongo_retriever.Document object at 0x72e5d9bfc190>, <zmongo_retriever.Document object at 0x72e5d9bfc1f0>, <zmongo_retriever.Document object at 0x72e5d9bfc250>, <zmongo_retriever.Document object at 0x72e5d9bfc2b0>, <zmongo_retriever.Document object at 0x72e5d9bfc310>], 'output_text': ' - Craig Lamb appealed a final judgment of foreclosure.\n   - The trial court erred in determining that Nationstar Mortgage, LLC had standing to bring a foreclosure action.\n   - The court reviewed the sufficiency of the evidence to prove standing to bring a foreclosure action de novo.\n   - The original note was specially indorsed to Aurora. \n   - Nationstar failed to prove its standing through evidence of an assignment, proof of purchase of the debt or an affidavit of ownership proving its status as holder of the note.\n 